# 1. Media Creation Example

This example shows how to create a media object from a file and upload it to the server. It save the metadata in the postgres db and uploads the files to the s3 bucket.

In [14]:
from rts.db.dao import DataAccessObject
from rts.db_settings import DATABASE_URL
from rts.db.queries import get_library_id_from_name, read_media_by_source_file_id
from rts.io.media import upload_media_files
from rts.api.models import Media
import os
import hashlib
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
DataAccessObject().connect(DATABASE_URL)

ARCHIVE_BASE_PATH = os.getenv("BASE_PATH")

In [2]:
# Setup before we can create media objects
archive_name =  "rts"
bucket_name =  archive_name
library_id =  get_library_id_from_name(archive_name)

In [3]:
# Upload clips

clips = []
data_dir = os.path.join(ARCHIVE_BASE_PATH, "data")
for dirname in os.listdir(data_dir):
    try:
        clip_dir = os.path.join(data_dir, dirname, 'clips', 'videos')

        for clip in os.listdir(clip_dir):
            clip_path = os.path.join(clip_dir, clip)

            clips.append(Media(**{
                    'original_path': clip_path,
                    'original_id': dirname,
                    'media_path': f"{bucket_name}/videos/{dirname}/{clip}", 
                    'media_type': "video",
                    'file_id': clip.split('.')[0],
                    'sub_type': "clip", 
                    'size': os.path.getsize(clip_path), 
                    'metadata': {},
                    'library_id': library_id, 
                    'hash': hashlib.md5(f"{bucket_name}/videos/{dirname}/{clip}".encode()).hexdigest(), 
                    'parent_id': -1,
                    'start_ts': 0, 'end_ts': 10, 
                    'start_frame': 0, 'end_frame': 10, 'frame_rate': 30, 
            }))

    except NotADirectoryError:
        pass
print(len(clips))

33


In [4]:
uploaded_clips = upload_media_files(clips)

In [21]:
# Upload thumbnails
thumbnails = []
square_res = 256

data_dir = os.path.join(ARCHIVE_BASE_PATH, "data")
for dirname in os.listdir(data_dir):
    try:
        clip_dir = os.path.join(data_dir, dirname, 'clips', 'images', '256px')

        for clip in os.listdir(clip_dir):
            clip_path = os.path.join(clip_dir, clip)

            # Find the parent. There can be multiple ways this can be achieved but
            # most of the time this will not be solely based with the database
            source_file_id = clip.split('.')[0][:-3]
            parent_id = read_media_by_source_file_id(source_file_id).media_id

            thumbnails.append(Media(**{
                    'original_path': clip_path,
                    'original_id': dirname,
                    'media_path': f"{bucket_name}/images/{dirname}/{square_res}px/{clip}", 
                    'media_type': "image", 
                    'file_id': clip.split('.')[0],
                    'sub_type': "thumbnail", 
                    'size': os.path.getsize(clip_path),
                    'metadata': {},
                    'library_id': library_id, 
                    'hash': hashlib.md5(f"{bucket_name}/videos/{dirname}/{clip}".encode()).hexdigest(), 
                    'parent_id': parent_id,  # Fill in the info here from the media objects
                    'start_ts': 0, 'end_ts': 10, 
                    'start_frame': 0, 'end_frame': 10, 'frame_rate': 30, 
            }))

    except NotADirectoryError:
        pass
print(len(thumbnails))

99


In [22]:
uploaded_clips = upload_media_files(thumbnails)